In [4]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
from random import randint
import random
from time import sleep
import re
import os

In [5]:
user_agent = {'User-Agent' : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.35"}

In [8]:
non_scraped_urls = []
list_of_ids = [5,15,51,200] #271

while(len(list_of_ids) != 0):
	comp_id = random.choice(list_of_ids) # get random competition from list
	url_comp = "https://www.zerozero.pt/competition.php?id_comp=" + str(comp_id) #builds the url
	list_of_ids.pop(list_of_ids.index(comp_id)) #removes the id from the list, so it doesn't double scrape
	#soup the response (main competition page and get the links to individual seasons)
	response = req.get(url_comp, headers=user_agent)
	assert(response.status_code == 200)
	soup = BeautifulSoup(response.content)
	links = soup.find_all("a", attrs={"style":"text-decoration:none;"})
	# scrape the competition name and create the corresponding dir
	dir_name = str(soup.find_all("span", attrs = {"style":"color:rgb(255,255,255) !important;"})[0].text)
	os.mkdir(dir_name)
	# register the competition in a file
	f = open("scraped_competitions.txt", "a+")
	f.write(str(comp_id) + " - " + dir_name + "\n")
	f.close()
	# Go through 20 links (starts in i + 4, so to avoid the last few seasons) and scrape each one
	for i in range(20):
		url_season = "https://www.zerozero.pt/" + links[i + 4]['href']
		print(url_season)
		season = req.get(url=url_season, headers=user_agent)
		assert(season.status_code == 200)
		soup = BeautifulSoup(season.content)
		# save the data
		names = [i.text for i in soup.find_all("td", attrs = {"class":"text"})]
		total_games = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos jogos disputados"})]
		wins = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todas as vitórias"})]
		ties = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os empates"})]
		losses = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todas as derrotas"})]
		points = [i.text for i in soup.find_all("strong")]
		goals_for = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os jogos com golos marcados"})]
		goals_against = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os jogos com golos sofridos"})]
		# checks for blocking (if goals_against isn't found - could be any other variable) and if blocked, records the url and waits 20-40m
		# this is also triggered if there's no league table, but there's no good solution for that
		if (len(goals_against) == 0):
			print(f"Reading failed. Break - {url_season} was NOT scraped")
			non_scraped_urls.append(url_season)
			f = open("non_scraped_urls.txt", "a+")
			f.write(url_season + '\n')
			f.close()
			delay = randint(1200, 2400)
			sleep(delay)
			print(f"Now sleeping for {round(delay/60, 2)} minutes, let's hope they unblock us soon")
		else:
			# if not blocked, add rank and team number, rearrange the columns and save the table (also displays for visual confirmation)
			table = pd.concat([pd.Series(data=names, name="team_name"), pd.Series(total_games, name="GP"), pd.Series(wins, name="W"), pd.Series(ties, name="D"), pd.Series(losses, name="L"), pd.Series(goals_for, name="GF"), pd.Series(goals_against, name="GA"), pd.Series(points, name="P")], axis=1)
			table["rank"] = np.arange(1,len(table) + 1)
			table["team_n"] = len(table)
			table = table.iloc[:, [8,0,1,2,3,4,5,6,7,9]]
			file_name = dir_name + "/" + re.findall('[0-9]+', url_season)[0]
			table.to_csv(file_name + ".csv")
			if ((comp_id == 15) | (comp_id == 271)):
				f = open(file_name + ".txt", "w+")
				f.write(season.text)
				f.close()
			display(table)
			# a 1 in 5 chance to access the main competition table, as to mimic human behaviour
			if randint(1,5) == 5:
				delay = randint(25,150)
				req.get(url_comp, headers=user_agent)
			# one in eight chance to take a bit more time (also to mimic human behaviour), but always wait at least 25 seconds before moving on
			if randint(1,8) == 8:
				delay = randint(120,240)
			else:
				delay = randint(25,150)
			print(f"Delaying for {delay} seconds...")
			sleep(delay)


https://www.zerozero.pt/edition.php?id=125020


,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Charlton Athletic,46,27,13,6,90,42,94,30
1,2,Sunderland,46,26,13,7,80,39,91,30
2,3,Sunderland,46,26,10,10,73,40,88,30
3,4,Portsmouth,46,25,13,8,83,51,88,30
4,5,Doncaster Rovers,46,22,19,5,80,47,85,30
5,6,Charlton Athletic,46,20,13,13,76,58,73,30
6,7,Luton Town,46,20,12,14,71,62,72,30
7,8,Barnsley,46,18,11,17,54,54,65,30
8,9,Charlton Athletic,46,17,12,17,66,57,63,30
9,10,Portsmouth,46,15,17,14,50,52,62,30


Delaying for 73 seconds...
https://www.zerozero.pt/edition.php?id=107513


,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Rotherham,46,29,11,6,89,29,98,30
1,2,Shrewsbury Town,46,28,12,6,82,40,96,30
2,3,Charlton Athletic,46,25,12,9,60,39,87,30
3,4,Shrewsbury Town,46,24,7,15,73,53,79,30
4,5,Scunthorpe United,46,19,17,10,65,50,74,30
5,6,Rotherham,46,20,11,15,58,51,71,30
6,7,Wigan Athletic,46,19,11,16,58,59,68,30
7,8,Blackburn Rovers,46,20,6,20,57,56,66,30
8,9,Shrewsbury Town,46,17,13,16,68,60,64,30
9,10,Rotherham,46,17,12,17,58,62,63,30


Delaying for 123 seconds...
https://www.zerozero.pt/edition.php?id=98049


...

,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Real Madrid,38,24,8,6,81,40,80,20
1,2,Deportivo,38,22,7,9,73,44,73,20
2,3,Mallorca,38,20,11,7,61,43,71,20
3,4,Barcelona,38,17,12,9,80,57,63,20
4,5,Valencia,38,18,9,11,55,34,63,20
5,6,Celta de Vigo,38,16,11,11,51,49,59,20
6,7,Villarreal,38,16,9,13,58,52,57,20
7,8,Málaga,38,16,8,14,60,61,56,20
8,9,Espanyol,38,14,8,16,46,44,50,20
9,10,Alavés,38,14,7,17,58,59,49,20


Delaying for 72 seconds...
https://www.zerozero.pt/edition.php?id=196


,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Deportivo,38,21,6,11,66,44,69,20
1,2,Barcelona,38,19,7,12,70,46,64,20
2,3,Valencia,38,18,10,10,59,39,64,20
3,4,Real Zaragoza,38,16,15,7,60,40,63,20
4,5,Real Madrid,38,16,14,8,58,48,62,20
5,6,Alavés,38,17,10,11,41,37,61,20
6,7,Celta de Vigo,38,15,8,15,45,43,53,20
7,8,Valladolid,38,14,11,13,36,44,53,20
8,9,Rayo Vallecano,38,15,7,16,51,53,52,20
9,10,Mallorca,38,14,9,15,52,45,51,20


Delaying for 138 seconds...


IndexError: list index out of range

In [9]:
url_comp

'https://www.zerozero.pt/competition.php?id_comp=15'

In [5]:
# snippet to scrape specific tables

dir_name = "Unprocessed"
f = open("non_scraped_urls.txt")
for url_season in f.readlines():
	print(url_season)
	season = req.get(url=url_season, headers=user_agent)
	assert(season.status_code == 200)
	soup = BeautifulSoup(season.content)
	names = [i.text for i in soup.find_all("td", attrs = {"class":"text"})]
	total_games = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos jogos disputados"})]
	wins = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todas as vitórias"})]
	ties = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os empates"})]
	losses = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todas as derrotas"})]
	points = [i.text for i in soup.find_all("strong")]
	goals_for = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os jogos com golos marcados"})]
	goals_against = [i.text for i in soup.find_all("a", attrs = {"title":"Ver todos os jogos com golos sofridos"})]
	if (len(goals_against) == 0):
		print(f"Reading failed. Break - {url_season} was NOT scraped")
		f = open("non_scraped_urls.txt", "a+")
		f.write(url_season + '\n')
		f.close()
		sleep(randint(1200, 2400))
	else:
		table = pd.concat([pd.Series(data=names, name="team_name"), pd.Series(total_games, name="GP"), pd.Series(wins, name="W"), pd.Series(ties, name="D"), pd.Series(losses, name="L"), pd.Series(goals_for, name="GF"), pd.Series(goals_against, name="GA"), pd.Series(points, name="P")], axis=1)
		table["rank"] = np.arange(1,len(table) + 1)
		table["team_n"] = len(table)
		table = table.iloc[:, [8,0,1,2,3,4,5,6,7,9]]
		file_name = dir_name + "/" + re.findall('[0-9]+', url_season)[0]
		table.to_csv(file_name + ".csv")
		# if ((comp_id == 15) | (comp_id == 271)):
		# 	f = open(file_name + ".txt", "w+")
		# 	f.write(season.text)
		# 	f.close()
		display(table)
		if randint(1,5) == 5:
			delay = randint(25,150)
			req.get(url_comp, headers=user_agent)
		if randint(1,8) == 8:
			delay = randint(120,240)
		else:
			delay = randint(25,150)
		print(f"Delaying for {delay} seconds...")
		sleep(delay)

https://www.zerozero.pt/edition.php?id=28683



,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Cheltenham Town,46,31,9,6,71,28,102,30
1,2,Rushden,46,30,7,9,96,48,97,30
2,3,Rushden,46,24,7,15,72,60,79,30
3,4,Rochdale,46,21,15,10,66,49,78,30
4,5,Hartlepool United,46,21,15,10,65,52,78,30
5,6,Cheltenham Town,46,20,13,13,69,53,73,30
6,7,Plymouth Argyle,46,20,11,15,74,48,71,30
7,8,Luton Town,46,19,14,13,74,56,71,30
8,9,Mansfield Town,46,20,10,16,64,53,70,30
9,10,Cheltenham Town,46,19,9,18,56,47,66,30


Delaying for 188 seconds...
https://www.zerozero.pt/edition.php?id=2156



,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Ajax,34,25,5,4,79,31,80,18
1,2,PSV,34,23,5,6,92,30,74,18
2,3,Feyenoord,34,20,8,6,71,38,68,18
3,4,Heerenveen,34,17,7,10,45,35,58,18
4,5,AZ Alkmaar,34,17,6,11,65,42,57,18
5,6,Roda,34,14,11,9,60,41,53,18
6,7,Willem II,34,13,10,11,47,54,49,18
7,8,FC Twente,34,15,3,16,56,53,48,18
8,9,NAC Breda,34,12,10,12,58,55,46,18
9,10,FC Utrecht,34,13,7,14,42,52,46,18


Delaying for 86 seconds...


In [3]:
f = open("non_scraped_urls.txt")
for line in f.readlines():
	print(line)
f.close()

https://www.zerozero.pt/edition.php?id=28683

https://www.zerozero.pt/edition.php?id=2156

